# 🔒 SCA Package Model Training - Google Colab (GitHub Dataset)

Train CodeLlama 7B to detect package vulnerabilities using 2024-2025 CVE data

**Before starting:**
1. Enable GPU: Runtime → Change runtime type → T4 GPU
2. Dataset will download automatically from GitHub
3. Run cells one by one with Shift+Enter

**Dataset:** 2024-2025 CVEs (~10,000 training examples, ~200MB)

## Step 1: Check GPU

In [1]:
import torch
print(f"🖥️  GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"📊 GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("❌ No GPU! Go to Runtime → Change runtime type → Select T4 GPU")

🖥️  GPU Available: True
📊 GPU Name: Tesla T4
💾 GPU Memory: 14.74 GB


## Step 2: Install Dependencies

In [2]:
%%capture
# Silent installation (remove %%capture to see output)
!pip install -q transformers==4.37.0 datasets==2.16.0 peft==0.8.0 accelerate==0.26.0 sentencepiece

print("✅ Dependencies installed (without bitsandbytes - using float16 instead)")

## Step 3: Download Dataset from GitHub

In [3]:
import os

# GitHub repository details
GITHUB_REPO = "abhay2510kr/ai_sec"
DATASET_FILE = "datasets/sca_training_2024_2025.json"
DATASET_URL = f"https://raw.githubusercontent.com/{GITHUB_REPO}/main/{DATASET_FILE}"

print(f"📥 Downloading dataset from GitHub...")
print(f"Repository: {GITHUB_REPO}")
print(f"File: {DATASET_FILE}")

# Download dataset
!wget -q --show-progress {DATASET_URL} -O /content/sca_training_dataset.json

# Verify download
if os.path.exists('/content/sca_training_dataset.json'):
    file_size = os.path.getsize('/content/sca_training_dataset.json') / (1024 * 1024)
    print(f"\n✅ Dataset downloaded successfully!")
    print(f"📊 File size: {file_size:.2f} MB")
else:
    print("\n❌ Download failed! Check if dataset exists in GitHub repo")
    print(f"URL: {DATASET_URL}")

📥 Downloading dataset from GitHub...
Repository: abhay2510kr/ai_sec
File: datasets/sca_training_2024_2025.json
/content/sca_traini 100%[===================>]  14.21M  --.-KB/s    in 0.1s    

✅ Dataset downloaded successfully!
📊 File size: 14.21 MB


## Step 4: Load and Prepare Dataset

In [4]:
from datasets import load_dataset

DATASET_PATH = "/content/sca_training_dataset.json"

print(f"📂 Loading dataset from: {DATASET_PATH}")

# Load dataset
dataset = load_dataset('json', data_files=DATASET_PATH)

# Split into train/validation
dataset = dataset['train'].train_test_split(test_size=0.1, seed=42)

print(f"\n📊 Dataset Statistics:")
print(f"  Training samples: {len(dataset['train'])}")
print(f"  Validation samples: {len(dataset['test'])}")

# Show sample
print(f"\n📝 Sample training example:")
print(dataset['train'][0]['text'][:600] + "...")

📂 Loading dataset from: /content/sca_training_dataset.json


Generating train split: 0 examples [00:00, ? examples/s]


📊 Dataset Statistics:
  Training samples: 9000
  Validation samples: 1000

📝 Sample training example:
<s>[INST] Analyze this npm dependency manifest for known vulnerabilities

```json
{
  "name": "vulnerable-app",
  "dependencies": {
    "Acrobat Reader": "0"
  }
}
``` [/INST]

Vulnerabilities found:

1. **CVE-2024-30302** (MEDIUM)
   - Affected packages: Acrobat Reader
   - Description: Acrobat Reader versions 20.005.30539, 23.008.20470 and earlier are affected by a Use After Free vulnerability that could lead to disclosure of sensitive memory. An attacker could leverage this vulnerability to bypass mitigations such as ASLR. Exploitation of this issue requires user interaction in t
   - Recom...


## Step 5A: Clear GPU Memory (Run this first if reloading model)

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print("📥 Loading CodeLlama-7b-Instruct...")
print("  Loading in float16 (no quantization)")

if torch.cuda.is_available():
    model = AutoModelForCausalLM.from_pretrained(
        "codellama/CodeLlama-7b-Instruct-hf",
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
    )
    print("  ✅ Model loaded in float16 (~14 GB)")
    print("  💡 Fits perfectly on T4 GPU (15GB)")
else:
    model = AutoModelForCausalLM.from_pretrained(
        "codellama/CodeLlama-7b-Instruct-hf",
        torch_dtype=torch.float32,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
    )
    print("  ✅ Model loaded on CPU")

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("\n✅ Model and tokenizer ready!")

📥 Loading CodeLlama-7b-Instruct...
  Loading in float16 (no quantization)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]


✅ Model and tokenizer ready!


## Step 5: Load Model (Float16 - No Quantization)

## Step 6: Configure LoRA (Train only 0.5% of parameters)

In [6]:
from peft import LoraConfig, get_peft_model

# Check if model was loaded in Step 5
try:
    model
except NameError:
    print("❌ ERROR: Model not loaded!")
    print("📌 Please run Step 5 first to load the model")
    raise NameError("Model not found. Run Step 5 (Load Model) before this cell.")

# LoRA configuration - this trains only 0.5% of parameters!
lora_config = LoraConfig(
    r=16,  # Rank (lower = less parameters, faster training)
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

print("ℹ️  Applying LoRA (training only 0.5% of model parameters)")
model = get_peft_model(model, lora_config)

# Show trainable parameters
model.print_trainable_parameters()
# Expected: trainable params: ~40M / 7B (~0.5%!)

ℹ️  Applying LoRA (training only 0.5% of model parameters)
trainable params: 16,777,216 || all params: 6,755,323,904 || trainable%: 0.24835546360798158


## Step 7: Tokenize Dataset

In [7]:
def tokenize_function(examples):
    # Tokenize the text
    result = tokenizer(
        examples["text"],
        truncation=True,
        max_length=2048,
        padding="max_length",
    )
    # For causal language modeling, labels are the same as input_ids
    result["labels"] = result["input_ids"].copy()
    return result

print("🔄 Tokenizing dataset...")

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

print("✅ Dataset tokenized!")

🔄 Tokenizing dataset...


Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

✅ Dataset tokenized!


## Step 8: Configure Training

In [8]:
from transformers import TrainingArguments, Trainer

# Check if GPU is available
use_fp16 = torch.cuda.is_available()

training_args = TrainingArguments(
    output_dir="/content/sca-package-checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,  # Effective batch size = 16
    learning_rate=2e-4,
    fp16=use_fp16,  # Only use fp16 if GPU available
    save_strategy="steps",
    save_steps=100,
    logging_steps=10,
    warmup_steps=50,
    optim="adamw_torch",  # Use standard optimizer (no 8bit)
    max_grad_norm=0.3,
    report_to="none",
    evaluation_strategy="steps",
    eval_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

print("✅ Trainer configured!")
if use_fp16:
    print(f"\n⏱️  Estimated training time: 4-6 hours on T4 GPU")
else:
    print(f"\n⏱️  Estimated training time: 24+ hours on CPU")
print(f"💾 Checkpoints will be saved to: /content/sca-package-checkpoints")
print(f"⚠️  Remember to download the model before session ends!")

✅ Trainer configured!

⏱️  Estimated training time: 4-6 hours on T4 GPU
💾 Checkpoints will be saved to: /content/sca-package-checkpoints
⚠️  Remember to download the model before session ends!


/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:449: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


## Step 9: START TRAINING! 🚀

In [9]:
print("🚀 Starting training...")
print("⏰ This will take 2-4 hours")
print("💡 TIP: You can close this tab - training will continue!")
print("\n" + "="*60)

trainer.train()

print("\n" + "="*60)
print("✅ Training complete!")
print("="*60)

🚀 Starting training...
⏰ This will take 2-4 hours
💡 TIP: You can close this tab - training will continue!



OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 8919 has 14.73 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 33.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Step 10: Save Final Model

In [ ]:
output_dir = "/content/sca-package-final"

print(f"💾 Saving final model to: {output_dir}")

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Model saved successfully!")
print(f"\n📁 Model location: {output_dir}")
print("\n⬇️  IMPORTANT: Download the model now!")
print("   Run the next cell to create a downloadable zip file")

In [ ]:
import shutil
from google.colab import files

# Create zip file
print("📦 Creating zip file...")
shutil.make_archive('/content/sca-package-final', 'zip', '/content/sca-package-final')

print("✅ Zip file created!")
print(f"📊 Size: {os.path.getsize('/content/sca-package-final.zip') / (1024*1024):.2f} MB")

# Download
print("\n⬇️  Starting download...")
files.download('/content/sca-package-final.zip')

print("\n✅ Download complete!")
print("💡 Save this file - you can use it to run inference later!")

## Step 11: Download Model (IMPORTANT!)

## Step 12: Test the Model! 🧪

In [ ]:
print("🧪 Testing the trained model...\n")

# Test input
test_input = """[INST] Analyze this package.json for known vulnerabilities

```json
{
  "name": "my-app",
  "dependencies": {
    "express": "4.16.0",
    "lodash": "4.17.4",
    "axios": "0.18.0"
  }
}
``` [/INST]"""

# Tokenize and generate
inputs = tokenizer(test_input, return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.1,
    do_sample=True,
    top_p=0.95
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("="*60)
print("🤖 MODEL OUTPUT:")
print("="*60)
print(result)
print("="*60)

## 🎉 Congratulations!

You've successfully trained your SCA model on 2024-2025 CVE data!

**What you have:**
- ✅ Trained model (downloaded as zip)
- ✅ Can detect package vulnerabilities
- ✅ Ready for production use

**Next Steps:**
1. Train more models: SAST, IaC, Container, etc.
2. Deploy the model using vLLM or Ollama
3. Integrate into your CI/CD pipeline

**Model Info:**
- Base: CodeLlama-7b-Instruct
- Training data: 2024-2025 CVEs (~10K examples)
- Training time: ~4-6 hours
- Model size: ~7GB (quantized)